# Language Model

## Initialization of Datasets

### Storing nGram Data

In [1]:
import pandas as pd

tl_ngram_data = pd.read_json('src/json data/Example-Based/Language Model/dict_tl_ngram.json')
il_ngram_data = pd.read_json('src/json data/Example-Based/Language Model/dict_il_ngram.json')

### Storing in lists

In [2]:
tl_ngram_list = tl_ngram_data['Tagalog nGram'].tolist()
il_ngram_list = il_ngram_data['Ilokano nGram'].tolist()

### Language Model Tagalog to Ilokano Dictionary

In [3]:
dict_tl_il_lang_mod = pd.DataFrame(columns=['Tagalog Structure', 'Ilokano Structure', 'Ilokano Structure Count'])
dict_tl_il_struct = pd.DataFrame(columns=['Tagalog Structure Count', 'Ilokano Structure Count'])

## Counter

In [4]:
def count_ngram(ngram_data):
    count_struct_list = []
    for ngram_sen in ngram_data:
        count_struct = []
        for capsule in ngram_sen:
            count_vb = 0
            count_nn = 0
            count_jj = 0
            count_pr = 0
            for pos in capsule:
                if pos == 'VB':
                    count_vb += 1
                if pos == 'NN':
                    count_nn += 1
                if pos == 'JJ':
                    count_jj += 1
                if pos == 'PR':
                    count_pr += 1
            
            count_struct.append([count_vb, count_nn, count_jj, count_pr])
        count_struct_list.append(count_struct)
            
    return count_struct_list

In [5]:
tl_count_struct_list = count_ngram(tl_ngram_data['Tagalog nGram'])
il_count_struct_list = count_ngram(il_ngram_data['Ilokano nGram'])

dict_tl_il_struct['Tagalog Structure Count'] = tl_count_struct_list
dict_tl_il_struct['Ilokano Structure Count'] = il_count_struct_list

dict_tl_il_struct.head()

,Tagalog Structure Count,Ilokano Structure Count
0,"[[0, 0, 0, 0]]","[[0, 0, 0, 0]]"
1,"[[1, 1, 0, 0], [0, 1, 0, 0]]","[[1, 1, 0, 0], [0, 1, 0, 0]]"
2,"[[1, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0,...","[[0, 1, 0, 0], [1, 1, 0, 0], [0, 1, 0, 0], [0,..."
3,"[[0, 0, 0, 0], [0, 1, 1, 0], [0, 0, 0, 0], [0,...","[[0, 0, 0, 0], [1, 1, 0, 0], [0, 1, 1, 0], [0,..."
4,"[[0, 0, 0, 0], [1, 1, 0, 0], [1, 1, 0, 0], [0,...","[[0, 0, 0, 0], [1, 1, 0, 0], [1, 1, 0, 0], [0,..."


## Matcher

In [6]:
def match_tl_il_lang_mod():
    sp_index = 0
    
    tl_struct_list = []
    il_struct_list = []
    
    tl_struct_count_list = []
    il_struct_count_list = []
    
    for tl_ngram_sen in tl_ngram_list:
        wp_index = 0
        
        tl_count_struct_sen = tl_count_struct_list[sp_index]
        il_count_struct_sen = il_count_struct_list[sp_index]
        
        il_ngram_sen = il_ngram_list[sp_index]
        
        for tl_ngram in tl_ngram_sen:
            tl_count_stuct = tl_count_struct_sen[wp_index]
            
            if len(tl_ngram_sen) == len(il_ngram_sen):
                curr_il_ngram = il_ngram_sen[wp_index]
                il_curr_stuct = il_count_struct_sen[wp_index]
                il_next_stuct = None
                il_prev_stuct = None
                # print('current_il_ngram:', curr_il_ngram)
                
                try:
                    next_il_ngram = il_ngram_sen[wp_index + 1]
                    il_next_stuct = il_count_struct_sen[wp_index + 1]
                    # print('next_il_ngram: ', next_il_ngram)
                except:
                    pass
                try:
                    if (wp_index - 1) != -1:
                        prev_il_ngram = il_ngram_sen[wp_index - 1]
                        il_prev_stuct = il_count_struct_sen[wp_index - 1]
                        # print('prev_il_ngram: ', prev_il_ngram)
                except:
                    pass
                
                """
                    Counter for the number of words in the nGram
                """
                
                # Stories with Tagalog nGram of in the DataFrame                   
                if tl_ngram not in tl_struct_list:
                    tl_struct_list.append(tl_ngram)
                    inDict = False
                else:
                    temp_index = tl_struct_list.index(tl_ngram)
                    inDict = True
                
                # Conditions for the current word    
                if tl_count_stuct == il_curr_stuct:    
                    il_gram = curr_il_ngram
                
                elif tl_count_stuct == il_next_stuct:
                    il_gram = next_il_ngram
                
                elif tl_count_stuct == il_prev_stuct:
                    il_gram = prev_il_ngram
                    
                else:
                    il_gram = tl_ngram
                
                # Storing the Ilokano nGram in the DataFrame    
                if not inDict:
                    il_struct_list.append([il_gram])
                    il_struct_count_list.append([1])
                else:
                    if il_gram not in il_struct_list[temp_index]:
                        il_struct_list[temp_index].append(il_gram)
                        il_struct_count_list[temp_index].append(1)
                    else:
                        temp_struct_index = il_struct_list[temp_index].index(il_gram)
                        il_struct_count_list[temp_index][temp_struct_index] += 1
            wp_index += 1
        sp_index += 1
        
    dict_tl_il_lang_mod['Tagalog Structure'] = tl_struct_list
    dict_tl_il_lang_mod['Ilokano Structure'] = il_struct_list
    dict_tl_il_lang_mod['Ilokano Structure Count'] = il_struct_count_list
            

In [7]:
match_tl_il_lang_mod()

dict_tl_il_lang_mod.head(20)

,Tagalog Structure,Ilokano Structure,Ilokano Structure Count
0,[SW],[[SW]],[8]
1,"[VB, DT, NN]","[[DT, VB, DT, NN], [DT, NN, VB], [VB, DT, NN],...","[99, 358, 857, 40, 72, 61]"
2,"[DT, NN]","[[DT, NN]]",[5041]
3,"[DT, VB, DT, NN]","[[DT, NN, VB], [DT, VB, DT, NN], [VB, DT, NN],...","[62, 177, 68, 15, 22, 18]"
4,[CC],"[[CC], [CC, RB]]","[3704, 3]"
5,"[DT, NN, DT, JJ]","[[DT, NN, DT, JJ], [DT, JJ, DT, NN], [DT, JJ, ...","[210, 13, 3, 9]"
6,"[VB, DT, JJ, NN]","[[VB, DT, JJ, NN]]",[59]
7,"[DT, NN, VB]","[[VB, DT, NN], [DT, NN, VB], [DT, NN, DT, VB],...","[75, 394, 17, 47, 36, 19]"
8,"[VB, NN]","[[VB, NN], [VB, DT, NN], [DT, NN, VB], [NN, VB...","[479, 259, 149, 79, 42, 18]"
9,"[RB, DT, VB]","[[RB, DT, VB]]",[99]


## Exporting the dictionary to a JSON File

In [8]:
import json

dict_tl_il_lang_mod_rec = dict_tl_il_lang_mod.to_dict('records')

try:
    with open('src/json data/Example-Based/Language Model/dict_tl_il_lang_mod.json', 'w') as outfile:
        json.dump(dict_tl_il_lang_mod_rec, outfile)
    print('Successfully saved the file!')
except:
    print('Error saving the file!')

Successfully saved the file!
